In [1]:
# The Base libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import laspy
import open3d as o3d
from plyfile import PlyData

# The Deep Learning libraries
import torch
from segment_anything import sam_model_registry
from segment_anything import SamAutomaticMaskGenerator

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
#selecting model checkpoint
MODEL = "1. Data/sam_vit_l_0b3195.pth"
#selecting gpu with cuda
USED_D = torch.device('cuda:0')

In [ ]:
def sam_masks(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    c_mask=[]
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*1)))
        c_mask.append(img)
    return c_mask

In [ ]:
def cloud_to_image(pcd_np, resolution):
    minx = np.min(pcd_np[:, 0])
    maxx = np.max(pcd_np[:, 0])
    miny = np.min(pcd_np[:, 1])
    maxy = np.max(pcd_np[:, 1])

    width = int((maxx - minx) / resolution) + 1
    height = int((maxy - miny) / resolution) + 1

    # Initialize empty image
    image = np.zeros((height, width, 3), dtype=np.uint8)

    # Compute pixel coordinates
    pixel_x = np.clip(((pcd_np[:, 0] - minx) / resolution).astype(int), 0, width - 1)
    pixel_y = np.clip(((maxy - pcd_np[:, 1]) / resolution).astype(int), 0, height - 1)

    # Assign color to the image
    image[pixel_y, pixel_x] = pcd_np[:, -3:].astype(np.uint8)

    return image

In [ ]:
# Read the point cloud with Open3D
pcd = PlyData.read("1. Data/SPLAT PLY_bouwpub cropped_inBlender.ply")

# Check if the file was loaded correctly
if len(pcd.points) == 0:
    print("Point cloud is empty. Ensure the file path and format are correct.")
else:
    print("Point cloud loaded successfully!")

In [ ]:
print(np.shape(orthoimage))


In [ ]:
def generate_spherical_image(center_coordinates, point_cloud, colors, resolution_y=500):
    # Translate the point cloud by the negation of the center coordinates
    translated_points = point_cloud - center_coordinates

    # Calculate norms once for efficiency
    norms = np.linalg.norm(translated_points, axis=1)
    
    # Convert 3D point cloud to spherical coordinates
    theta = np.arctan2(translated_points[:, 1], translated_points[:, 0])
    z_normalized = np.clip(translated_points[:, 2] / norms, -1.0, 1.0)
    phi = np.arccos(z_normalized)

    # Map spherical coordinates to pixel coordinates
    resolution_x = 2 * resolution_y
    x = (theta + np.pi) / (2 * np.pi) * resolution_x
    y = phi / np.pi * resolution_y

    # Create the spherical image with RGB channels
    image = np.zeros((resolution_y, resolution_x, 3), dtype=np.uint8)

    # Create the mapping between point cloud and image coordinates
    mapping = np.full((resolution_y, resolution_x), -1, dtype=int)

    # Assign points to the image pixels
    for i in range(len(translated_points)):
        ix = np.clip(int(x[i]), 0, resolution_x - 1)
        iy = np.clip(int(y[i]), 0, resolution_y - 1)
        # Update pixel if it's empty or closer than the current point
        if mapping[iy, ix] == -1 or norms[i] < norms[mapping[iy, ix]]:
            mapping[iy, ix] = i
            image[iy, ix] = colors[i]
    
    return image, mapping


In [ ]:
ply_data = PlyData.read("1. Data/polycam_aligned.ply")

# Extracting the coordinates (x, y, z)
coords = np.vstack((ply_data['vertex']['x'], ply_data['vertex']['y'], ply_data['vertex']['z'])).T



# Extracting the colors (r, g, b) if they are available
if {'red', 'green', 'blue'} <= set(ply_data['vertex'].data.dtype.names):
    r = ply_data['vertex']['red'].astype(int)
    g = ply_data['vertex']['green'].astype(int)
    b = ply_data['vertex']['blue'].astype(int)
    colors = np.vstack((r, g, b)).T
else:
    # If there are no colors, assign a default white color
    colors = np.ones((coords.shape[0], 3)) * 255

point_cloud = coords



In [ ]:
resolution = 1000

#Defining the position in the point cloud to generate a panorama
center_coordinates = [3.128129, 5, -6.0]


In [ ]:
#Function Execution
spherical_image, mapping = generate_spherical_image(center_coordinates, point_cloud, colors, resolution)


#Plotting with matplotlib for front
fig = plt.figure(figsize=(np.shape(spherical_image)[1]/72, np.shape(spherical_image)[0]/72))
fig.add_axes([0,0,1,1])
plt.imshow(spherical_image)
plt.axis('off')

#Saving to the disk
plt.savefig("7. Results/BUILDING_spherical_projection_fromsplat.png", dpi = 72)


In [ ]:
sam = sam_model_registry["vit_l"](checkpoint = MODEL)
sam.to(device = USED_D)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=64,  # Higher resolution for better detail due to the complex features.
    pred_iou_thresh=0.9,  # High threshold to ensure only high-confidence masks are kept.
    stability_score_thresh=0.95,  # High stability to maintain consistent boundaries, especially for architectural elements.
    crop_n_layers=1,  # One layer, as the spherical projection may not need multiple layers for accurate segmentation.
    crop_n_points_downscale_factor=2,  # Standard setting; balances resolution and performance.
    min_mask_region_area=5625  # Slightly larger to avoid noise and capture meaningful structures like windows or signs.
)

temp_img = cv2.imread("7. Results/BUILDING_spherical_projection_fromsplat.png")
image_rgb = cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB)


result = mask_generator.generate(image_rgb)


In [ ]:
#Plotting it back to img
fig = plt.figure(figsize=(np.shape(image_rgb)[1]/72, np.shape(image_rgb)[0]/72))
fig.add_axes([0,0,1,1])

plt.imshow(image_rgb, alpha = 0)
color_mask = sam_masks(result)
plt.axis('off')
plt.savefig("7. Results/BUILDING_spherical_projection_segmented_fromsplat.png", dpi = 72, bbox_inches='tight', pad_inches=0)


In [ ]:

image_path = "7. Results/BUILDING_spherical_projection_segmented.png"
def color_point_cloud(image_path, point_cloud, mapping):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    # Create an empty array to store the modified point cloud
    modified_point_cloud = np.zeros((point_cloud.shape[0], point_cloud.shape[1] + 3), dtype=np.float32)
    modified_point_cloud[:, :3] = point_cloud  # Copy the original point cloud

    for iy in range(h):
        for ix in range(w):
            # Clip ix and iy to ensure they don't exceed the bounds
            ix_clipped = np.clip(ix, 0, mapping.shape[1] - 1)
            iy_clipped = np.clip(iy, 0, mapping.shape[0] - 1)

            point_index = mapping[iy_clipped, ix_clipped]

            if point_index != -1:  # Ensure the point is valid
                color = image[iy_clipped, ix_clipped]
                modified_point_cloud[point_index, 3:] = color  # Assign color to the point cloud
                # print(f"Assigned color {color} to point index {point_index}")

    return modified_point_cloud

In [ ]:
modified_point_cloud = color_point_cloud(image_path, point_cloud, mapping)

In [ ]:
def export_point_cloud(cloud_path, modified_point_cloud):
    # 1. Create a new header
    header = laspy.LasHeader(point_format=3, version="1.2")
    header.add_extra_dim(laspy.ExtraBytesParams(name="random", type=np.int32))

    # 2. Create a Las
    las_o = laspy.LasData(header)
    las_o.x = modified_point_cloud[:,0]
    las_o.y = modified_point_cloud[:,1]
    las_o.z = modified_point_cloud[:,2]
    las_o.red = modified_point_cloud[:,3]
    las_o.green = modified_point_cloud[:,4]
    las_o.blue = modified_point_cloud[:,5]
    las_o.write(cloud_path)
    
    print("Export succesful at: ", cloud_path)
    return

In [ ]:
export_point_cloud("7. Results/pcd_results_SPLATTED.las", modified_point_cloud)

In [ ]:
def add_color_label_to_ply(las_file_path, ply_file_path, output_ply_file_path):
    # Step 1: Read LAS file and get the color labels
    with laspy.open(las_file_path) as las_file:
        las_data = las_file.read()
        color_labels = las_data.color_label  # Assuming you have already inserted this column

    # Step 2: Read PLY file
    ply_data = PlyData.read(ply_file_path)
    vertices = ply_data['vertex'].data

    # Ensure the number of points matches between the LAS and PLY files
    if len(vertices) != len(color_labels):
        raise ValueError("The number of points in the LAS file does not match the PLY file.")

    # Step 3: Add the labels as a new column to the PLY data
    new_dtype = vertices.dtype.descr + [('color_label', 'i4')]
    new_vertices = np.empty(vertices.shape, dtype=new_dtype)

    for name in vertices.dtype.names:
        new_vertices[name] = vertices[name]
    new_vertices['color_label'] = color_labels

    # Step 4: Write the modified PLY file in binary format
    new_ply_element = PlyElement.describe(new_vertices, 'vertex')
    PlyData([new_ply_element], text=False).write(output_ply_file_path)

    print(f"PLY file with color labels saved in binary format to: {output_ply_file_path}")

# File paths
las_file_path = '1. Data/pcd_results_SPLATTED.las'
ply_file_path = '1. DataSPLAT PLY_bouwpub cropped_inBlender-Copy1.ply'
output_ply_file_path = '7. Results/SAMSPLAT.ply'
add_color_label_to_ply(las_file_path, ply_file_path, output_ply_file_path)